In [8]:
%%capture
!apt-get install pandoc texlive-xetex texlive-fonts-recommended texlive-plain-generic

In [9]:
%%capture
!pip install nbconvert

In [10]:
%%capture
!jupyter nbconvert --to pdf --LatexPreprocessor.title "Assigment 2" --LatexPreprocessor.author_names Joao\ Gabriel --LatexPreprocessor.author_names Fahime --LatexPreprocessor.date "May 21th 2025" spark-task.ipynb

# Importing packages

Here we import the pyspark packages (installed with pip) which is necessary to access the spark funtions using python.


In [1]:
from pyspark import SparkContext

# Spark Context

* pyspark.SparkContext is an entry point to the PySpark. It creates the access to the SparkContext within the pySpark shell.

* The SparkContext is the main entry point for Spark. A SparkContext represents the connection to a Spark cluster and lets you interact with them using the spark functionalities.

In [2]:
sc = SparkContext("local", "spark-task")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/22 09:51:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Task-1 (a)

Start by creating a list of 0 to 40000, afterwards use parallelize() to make an RDD
from the list (distribute your list into four partitions). Use RRD’s filter and a
lambda function to convert your list into a list of odd numbers. Show the result by
using the take() function and output the first seven numbers in the odds list.

* The parallelize function distributes a local Python collection to form an RDD.

* An Resilient Distributed Dataset (RDD) is an essential structure within Spark.

* It represents an immutable distributed collection of elements of your data partitioned across nodes of the spark cluster.

* The filter function from the RDD creates a new RDD by selecting the elements that satisfy a given condition.

* In this case the condition is given by the lamda funtion.

* The Spark cluster is a collection of processes (workers) that execute functionalities from Spark to the RDD in pararallel.

In [3]:
sparkList = list(range(0,40001))
print("list type: ", type(sparkList[0]))
rdd = sc.parallelize(sparkList)
odd_rdd = rdd.filter(lambda x: x%2 == 1)
odd_rdd.take(7)

list type:  <class 'int'>


[1, 3, 5, 7, 9, 11, 13]

## Task-1(b)

Create an RDD with 10,000 random integers between 1 and 100. Partition the RDD into 5 partitions. Count how many numbers fall into each of the following buckets: 0-20, 21-40, 41-60, 61-80, 81-100. Show the results as key-value pairs: (”1-20”, count), (”21-40”, count), ...

* In this taks the RDD is created using a predefined number of buckets - 5.

* The buckets partition the RDD in 5 sections in order to parallelize the RDD functionalities using them in parallel.

* Here we use the map function to map each number within the category needed.

* Then we use reduceByKey in order to reduce the numbers in groups of each category.

In [4]:
import random
random_list = [random.randint(1,100) for _ in range(10000)]
random_rdd = sc.parallelize(random_list, 5)
def bucket(num):

    if 1<= num <= 20:
        return "1-20"
    elif 21<= num <= 40:
        return "21-40"
    elif 41<= num <= 60:
        return "41-60"
    elif 61<= num <= 80:
        return "61-80"
    elif 81<= num <= 100:
        return "81-100"
    else:
        return "OutOfRange"

counts = (random_rdd.map(lambda x: (bucket(x),1)).reduceByKey(lambda a,b: a+b))

res = counts.collect()

for bucket_name, count in sorted(res):
    print(f"({bucket_name}, {count})")

(1-20, 2018)
(21-40, 2007)
(41-60, 1989)
(61-80, 1954)
(81-100, 2032)


## Task-1 (c):

Explain what the parallelize() function does. Define RDD and describe the process
used step-by-step in part (a). After that, compare this approach with filtering even
numbers in a list using Python’s built-in filter() function (as done in the previous
assignment). Highlight the differences between using RDD’s filter() and Python’s
filter() in terms of execution and scalability.

* The parallelize function distributes a local Python collection to form an RDD.

* An Resilient Distributed Dataset (RDD) is an essential structure within Spark.

* It represents an immutable distributed collection of elements of your data partitioned across nodes of the spark cluster.

* The filter function from the RDD creates a new RDD by selecting the elements that satisfy a given condition.

* In this case the condition is given by the lamda funtion.

* The Spark cluster is a collection of processes (workers) that execute functionalities from Spark to the RDD in pararallel.

* The difference between the python filter and the Spark filter is that Spark parallelize this functionality.

* To do this, Spark works with several partitions of the entire data and run the funtionalities across the partitions at the same time.

* This makes it possible to work with big datasets without having to process the entire data everytime a functinonality is called.

## Task-1(d)

Count the number of appearances of each word in the text file ”Text-Data” using PySpark. Show the results of the 25 words with the most occurrences as a tuple (e.g., (”word”, 23)).

* Here we use a flat map the text into words.

* The regular expression returns only words without punctuation and in lower case.

* Then a map function is used to categorize data with the same value (repeated words).

* Then a reduceByKey function count how many values fall in the same category.

* Finally, the 25 most repeated words are taken by a takeOrdered function, that takes 25 groups and its cocunt value after ordering them in relation to their count value.


In [5]:
import re
text_rdd = sc.textFile("Text-data.txt")
words_rdd = text_rdd.flatMap(lambda line: re.findall(r'\b\w+\b', line.lower())) # Split each line into words --> no punctuation, lower case

# I need word-pairs. Use map function
words_pairs = words_rdd.map(lambda x: (x,1))

# sum to same words. Use ReduceByKey()

same_words = words_pairs.reduceByKey(lambda a,b: a+b)

# 25 most common words
top_25 = same_words.takeOrdered(25, key=lambda x: -x[1]) 
# If we didn't delete the punctuation mark, this line would give an error.

for word, count in top_25:
    print((word, count))

('is', 144276)
('the', 78015)
('a', 77125)
('of', 70999)
('to', 43196)
('or', 35443)
('in', 31537)
('and', 27278)
('an', 14550)
('that', 13839)
('with', 13356)
('by', 11357)
('for', 11048)
('on', 7501)
('as', 6783)
('from', 6782)
('who', 6003)
('having', 5808)
('he', 4824)
('used', 4803)
('was', 4359)
('his', 3990)
('one', 3808)
('s', 3683)
('at', 3630)


## Task-1 (e)

List all those words, which can be classified as ”auxiliary verbs” or ”function words” to exclude them from your results (e.g. ”by”, ”to”, ”and” are ”not-meaningful-words”). Order each ”meaningful” word by occurrence.

* First we create a list of words that we consider as ”auxiliary verbs” or ”function words”.

* Then we filter the RDD by excluding these words.

* After filtering we count how many meaningful words there are by using takeOrdered again.

* Finally, we print the 25 most counted meaninful words.

In [6]:
# set stopwords

stop_words = {
    "a", "an", "the", "and", "or", "but", "if", "while", "because", "as", "until", "of",
    "at", "by", "for", "with", "about", "against", "between", "into", "through", "during",
    "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off",
    "over", "under", "again", "further", "then", "once", "here", "there", "when", "where",
    "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some",
    "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very",
    "can", "will", "just", "don", "should", "now", "is", "am", "are", "was", "were",
    "be", "been", "being", "have", "has", "had", "do", "does", "did", "would", "could",
    "may", "might", "must", "shall", "also", "who", "whom", "this", "that", "these", "those", "you"
}

filtered_words = words_rdd.filter(lambda x: x not in stop_words)
meaningful_words_count = filtered_words.map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b)
top_25_meaningful_words = meaningful_words_count.takeOrdered(25, key=lambda x: -x[1])
for meaningful_word, count in top_25_meaningful_words:
    print((meaningful_word, count))

('having', 5808)
('he', 4824)
('used', 4803)
('his', 3990)
('one', 3808)
('s', 3683)
('small', 3145)
('genus', 3121)
('which', 2835)
('united', 2789)
('states', 2749)
('relating', 2666)
('it', 2613)
('something', 2335)
('large', 2223)
('person', 2214)
('she', 2189)
('manner', 1968)
('her', 1966)
('two', 1866)
('made', 1843)
('act', 1822)
('someone', 1812)
('its', 1731)
('north', 1666)


## Stop SparkContext

Finally we realease the sparkContext since we are not going to use it anymore

In [7]:
sc.stop()